In [6]:
import numpy as np
import nltk as nl
from nltk.tokenize import sent_tokenize,word_tokenize
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer as wnl

In [7]:
import pandas as pd
df = pd.read_csv("/Users/prajualpillai/Desktop/work/archive/olist_order_reviews_dataset.csv")
df.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [8]:
sen = df[df["review_comment_title"].isnull()==False]
sen.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
9,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,recomendo,aparelho eficiente. no site a marca do aparelh...,2018-05-22 00:00:00,2018-05-23 16:45:47
15,3948b09f7c818e2d86c9a546758b2335,e51478e7e277a83743b6f9991dbfa3fb,5,Super recomendo,"Vendedor confiável, produto ok e entrega antes...",2018-05-23 00:00:00,2018-05-24 03:00:01
19,373cbeecea8286a2b66c97b1b157ec46,583174fbe37d3d5f0d6661be3aad1786,1,Não chegou meu produto,Péssimo,2018-08-15 00:00:00,2018-08-15 04:10:37
22,d21bbc789670eab777d27372ab9094cc,4fc44d78867142c627497b60a7e0228a,5,Ótimo,Loja nota 10,2018-07-10 00:00:00,2018-07-11 14:10:25
34,c92cdd7dd544a01aa35137f901669cdf,37e7875cdce5a9e5b3a692971f370151,4,Muito bom.,Recebi exatamente o que esperava. As demais en...,2018-06-07 00:00:00,2018-06-09 18:44:02


In [9]:
title = sen["review_comment_title"].reset_index(drop=True)

In [10]:
len(set(title))

4600

In [11]:
len(title)

11715

In [12]:
title.unique()

array(['recomendo', 'Super recomendo', 'Não chegou meu produto ', ...,
       'Não foi entregue o pedido', 'OTIMA EMBALAGEM', 'Foto enganosa '],
      dtype=object)

In [13]:
sen.isnull().sum()

review_id                     0
order_id                      0
review_score                  0
review_comment_title          0
review_comment_message     1729
review_creation_date          0
review_answer_timestamp       0
dtype: int64

In [14]:
mess = df[df["review_comment_message"].isnull()==False]
mess.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53
9,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,recomendo,aparelho eficiente. no site a marca do aparelh...,2018-05-22 00:00:00,2018-05-23 16:45:47
12,4b49719c8a200003f700d3d986ea1a19,9d6f15f95d01e79bd1349cc208361f09,4,NaN,"Mas um pouco ,travando...pelo valor ta Boa.\r\n",2018-02-16 00:00:00,2018-02-20 10:52:22
15,3948b09f7c818e2d86c9a546758b2335,e51478e7e277a83743b6f9991dbfa3fb,5,Super recomendo,"Vendedor confiável, produto ok e entrega antes...",2018-05-23 00:00:00,2018-05-24 03:00:01


In [15]:
mess1 = mess[["order_id","review_comment_message","review_score"]].copy().reset_index(drop=True)

In [16]:
mess1

,order_id,review_comment_message,review_score
0,658677c97b385a9be170737859d3511b,Recebi bem antes do prazo estipulado.,5
1,8e6bfb81e283fa7e4f11123a3fb894f1,Parabéns lojas lannister adorei comprar pela I...,5
2,b9bf720beb4ab3728760088589c62129,aparelho eficiente. no site a marca do aparelh...,4
3,9d6f15f95d01e79bd1349cc208361f09,"Mas um pouco ,travando...pelo valor ta Boa.\r\n",4
4,e51478e7e277a83743b6f9991dbfa3fb,"Vendedor confiável, produto ok e entrega antes...",5
...,...,...,...
41748,509b86c65fe4e2ad5b96408cfef9755e,Entregou dentro do prazo. O produto chegou em ...,5
41749,d5cb12269711bd1eaf7eed8fd32a7c95,"O produto não foi enviado com NF, não existe v...",3
41750,55d4004744368f5571d1f590031933e4,"Excelente mochila, entrega super rápida. Super...",5
41751,f8bd3f2000c28c5342fedeb5e50f2e75,Solicitei a compra de uma capa de retrovisor c...,1


In [17]:
mess2 = mess1["review_comment_message"].copy()
score = mess1["review_score"].copy()

In [18]:
mess2

0                    Recebi bem antes do prazo estipulado.
1        Parabéns lojas lannister adorei comprar pela I...
2        aparelho eficiente. no site a marca do aparelh...
3          Mas um pouco ,travando...pelo valor ta Boa.\r\n
4        Vendedor confiável, produto ok e entrega antes...
                               ...                        
41748    Entregou dentro do prazo. O produto chegou em ...
41749    O produto não foi enviado com NF, não existe v...
41750    Excelente mochila, entrega super rápida. Super...
41751    Solicitei a compra de uma capa de retrovisor c...
41752    meu produto chegou e ja tenho que devolver, po...
Name: review_comment_message, Length: 41753, dtype: object

In [19]:
port = PorterStemmer()
wnet = wnl()

In [20]:
mess3 = []
for i in mess2:
    w = ""
    i = str.lower(i)
#
    x = word_tokenize(i)
    #print(x)
    for word in x:
        res = re.sub(r'[^\w\s]',"",word)
        if res != "":
            if res not in stopwords.words('portuguese'):
                stem = port.stem(res)
                lem = wnet.lemmatize(stem)
                w+=" "+lem
    mess3.append(w)


In [21]:
mess3

[' recebi bem ant prazo estipulado',
 ' parabén loja lannist adorei comprar internet seguro prático parabén todo feliz páscoa',
 ' aparelho eficient site marca aparelho impresso 3desinfector chegar outro nome atualizar marca correta vez aparelho',
 ' pouco travando valor ta boa',
 ' vendedor confiável produto ok entrega ant prazo',
 ' gostaria saber sempr recebi compra agora decpcion',
 ' péssimo',
 ' loja nota 10',
 ' obrigado atençao amim dispensada',
 ' compra realizada facilment entrega efetuada ant prazo dado produto começ ser usado present problema',
 ' relógio bonito barato',
 ' gostei comprei gato lebr',
 ' sempr compro internet entrega ocorr ant prazo combinado acredito ser prazo máximo stark prazo máximo esgot ainda recebi produto',
 ' recebi exatament esperava demai encomenda outro vendedor atrasaram chegou prazo',
 ' recomendo',
 ' boa',
 ' tô completament apaixonada loja super responsável confiável',
 ' nada chegar pedido',
 ' bom cheiroso',
 ' otimo vendedor chegou ate an

In [17]:
mess4 = pd.DataFrame(mess3,columns = ["message"])

In [40]:
score = mess1["review_score"].astype("category").cat.codes
score

0        4
1        4
2        3
3        3
4        4
        ..
41748    4
41749    2
41750    4
41751    0
41752    0
Length: 41753, dtype: int8

LabelEncoder()

In [41]:
x = mess3
y = score

In [50]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=1)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
tk = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [51]:
from transformers import AutoModelForPreTraining
model = AutoModelForPreTraining.from_pretrained('neuralmind/bert-base-portuguese-cased')
tk = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)

loading configuration file https://huggingface.co/neuralmind/bert-base-portuguese-cased/resolve/main/config.json from cache at /Users/prajualpillai/.cache/huggingface/transformers/e716e2151985ba669e7197b64cdde2552acee146494d40ffaf0688a3f152e6ed.18a0b8b86f3ebd4c8a1d8d6199178feae9971ff5420f1d12f0ed8326ffdff716
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "ab

In [68]:
x_tr = tk(X_train,truncation=True,padding=True,max_length=100)
x_tst = tk(X_test,truncation=True,padding=True,max_length=100)

In [ ]:
x_tr

In [70]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [71]:
train_dataset = IMDbDataset(x_tr, y_train)
#val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(x_tst, y_test)

In [63]:
train_dataset[0]

{'input_ids': tensor([  101,  2663, 22283,  3576,  8705, 22283,  2080,  1342,  2519,  2892,
           990,   129, 11631,  4270,  4938, 11289,  7841, 18253, 17512,   367,
          1988,  1434,  4708, 20780, 22280,  4862,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask':

In [75]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=10,  # batch size per device during training
    per_device_eval_batch_size=10,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

#model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
  0%|          | 0/879 [17:23<?, ?it/s]
***** Running training *****
  Num examples = 29227
  Num Epochs = 3
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 10
  Gradient Accumulation steps = 1
  Total optimization steps = 8769
  0%|          | 0/8769 [00:00<?, ?it/s]

KeyError: 22985

In [46]:
train_dataset